# 27 - Zeppelin jako Data Science Hub

Apache Zeppelin to webowy notebook do interaktywnej analizy danych. W odroznieniu od Jupytera, oferuje natywna integracje ze Spark, SQL, wieloma interpreterami w jednym notebooku i wbudowane wizualizacje.

**Tematy:**
- Architektura Zeppelin: Interpreter, Note, Paragraph
- Zeppelin REST API - zarzadzanie z Pythona
- Multi-interpreter: %spark, %sql, %python, %md, %sh
- Spark interpreter: konfiguracja i uzycie
- Wizualizacje w Zeppelin: wbudowane wykresy, Angular display
- Dynamic forms: parametryzowanie notebookow
- Porownanie: Zeppelin vs Jupyter
- Zeppelin + HDFS + Spark: analiza MovieLens
- Wspolpraca: note sharing, permissions

## 1. Architektura Apache Zeppelin

```
  +--------------------------------------------------+
  |              Zeppelin Web UI (:8085)              |
  |  +--------+ +--------+ +--------+ +--------+    |
  |  | Note 1 | | Note 2 | | Note 3 | | Note N |    |
  |  +--------+ +--------+ +--------+ +--------+    |
  +------------------------+-------------------------+
                           |
  +------------------------v-------------------------+
  |              Zeppelin Server                     |
  |  +-------------------------------------------+  |
  |  |          Interpreter Manager              |  |
  |  +-----+-------+-------+-------+------+-----+  |
  |        |       |       |       |      |         |
  |  +-----v-+ +---v---+ +v-----+ +v---+ +v----+   |
  |  |%spark | |%sql   | |%python| |%sh | |%md  |   |
  |  |       | |       | |       | |    | |     |   |
  |  |SparkS.| |SparkS.| |IPython| |bash| |mark.|   |
  |  +---+---+ +---+---+ +--+----+ +----+ +-----+   |
  +------+----------+-------+------------------------+
         |          |       |
    +----v----+ +---v--+ +--v------+
    |  Spark  | | HDFS | |PostgreSQL|
    | Cluster | |      | |         |
    +---------+ +------+ +---------+
```

### Kluczowe koncepty:
- **Note**: odpowiednik notebooka - dokument z paragrafami
- **Paragraph**: pojedyncza komorka (kod + wynik), kazdy moze uzywac innego interpretera
- **Interpreter**: backend wykonujacy kod (%spark, %sql, %python, %md, %sh)
- **Interpreter Group**: wspoldzielony kontekst (np. %spark i %sql wspoldziela SparkSession)
- **Angular Display**: dwukierunkowy binding UI <-> backend

## 2. Setup - polaczenie z Zeppelin REST API

Zeppelin udostepnia REST API do programowego zarzadzania notebookami.

In [ ]:
import requests
import json
import time

ZEPPELIN_URL = "http://zeppelin:8085"
HEADERS = {"Content-Type": "application/json"}


class ZeppelinClient:
    """Klient do komunikacji z Apache Zeppelin REST API."""

    def __init__(self, base_url):
        self.base_url = base_url
        self.session = requests.Session()
        self.session.headers.update(HEADERS)

    def login(self, user="admin", password="password"):
        resp = self.session.post(f"{self.base_url}/api/login",
                                 data={"userName": user, "password": password})
        if resp.status_code == 200:
            print(f"Zalogowano jako {user}")
        return resp.status_code

    def get(self, endpoint):
        resp = self.session.get(f"{self.base_url}/api{endpoint}")
        resp.raise_for_status()
        return resp.json()

    def post(self, endpoint, data=None):
        resp = self.session.post(f"{self.base_url}/api{endpoint}", json=data)
        resp.raise_for_status()
        return resp.json()

    def put(self, endpoint, data=None):
        resp = self.session.put(f"{self.base_url}/api{endpoint}", json=data)
        resp.raise_for_status()
        return resp.json()

    def delete(self, endpoint):
        resp = self.session.delete(f"{self.base_url}/api{endpoint}")
        resp.raise_for_status()
        return resp.json()


zep = ZeppelinClient(ZEPPELIN_URL)

try:
    version = zep.get("/version")
    print(f"Zeppelin version: {version.get('body', {}).get('version', 'unknown')}")
    print(f"Status: Connected")
except requests.exceptions.ConnectionError:
    print("UWAGA: Zeppelin nie jest dostepny. Uruchom kontener Zeppelin.")
    print("Notebook mozna czytac jako material edukacyjny.")

## 3. Zarzadzanie Interpreterami

Kazdy interpreter w Zeppelin ma swoja konfiguracje. Najwazniejsze:

| Interpreter | Prefix | Opis | Backend |
|------------|--------|------|--------|
| `%spark` | Scala | Spark (Scala API) | SparkSession |
| `%spark.pyspark` | Python | PySpark | SparkSession |
| `%spark.sql` / `%sql` | SQL | Spark SQL | SparkSession |
| `%python` | Python | Czysty Python (IPython) | IPython kernel |
| `%md` | Markdown | Dokumentacja | Zeppelin renderer |
| `%sh` | Shell | Bash | System shell |
| `%jdbc` | SQL | JDBC (PostgreSQL) | JDBC driver |

Kluczowa roznica vs Jupyter: w jednym notebooku mozna mieszac interpretery!

In [ ]:
# Listowanie dostepnych interpreterow
try:
    interpreters = zep.get("/interpreter/setting")
    print("Dostepne interpretery:")
    for interp in interpreters.get("body", []):
        name = interp.get("name", "?")
        group = interp.get("group", "?")
        status = interp.get("status", "?")
        print(f"  {name:<20} group={group:<15} status={status}")
except Exception as e:
    print(f"Blad: {e}")
    print("\nPrzykladowe interpretery w Zeppelin:")
    for name, desc in [
        ("spark", "Apache Spark (Scala, PySpark, SparkSQL, SparkR)"),
        ("python", "Python / IPython"),
        ("jdbc", "JDBC (PostgreSQL, MySQL, etc.)"),
        ("sh", "Shell / Bash"),
        ("md", "Markdown renderer"),
        ("angular", "Angular display system")
    ]:
        print(f"  {name:<20} {desc}")

In [ ]:
# Konfiguracja Spark interpretera dla MovieLens
spark_config = {
    "spark.master": "spark://spark-master:7077",
    "spark.app.name": "Zeppelin_MovieLens",
    "spark.driver.memory": "6g",
    "spark.executor.memory": "7g",
    "spark.jars.packages": "org.postgresql:postgresql:42.7.1",
    "spark.driver.host": "zeppelin",
    "spark.driver.bindAddress": "0.0.0.0",
    "zeppelin.spark.useHiveContext": "true",
    "zeppelin.spark.maxResult": "10000",
    "zeppelin.pyspark.python": "python3"
}

print("Konfiguracja Spark interpretera:")
for key, value in spark_config.items():
    print(f"  {key} = {value}")

# Aktualizacja interpretera przez API
try:
    settings = zep.get("/interpreter/setting")
    for interp in settings.get("body", []):
        if interp.get("name") == "spark":
            interp_id = interp["id"]
            props = interp.get("properties", {})
            for k, v in spark_config.items():
                props[k] = {"value": v, "type": "string"}
            interp["properties"] = props
            zep.put(f"/interpreter/setting/{interp_id}", interp)
            print("\nSpark interpreter zaktualizowany!")
            break
except Exception as e:
    print(f"\nAktualizacja interpretera (wymaga Zeppelin): {e}")

## 4. Multi-Interpreter - sila Zeppelin

W Zeppelin kazdy paragraf moze uzywac innego interpretera. Oto przyklad notebooka ktory laczy rozne narzedzia.

Ponizej pokazujemy zawartosc paragrafow tak, jak wygladajalyby w Zeppelin.

In [ ]:
# Symulacja paragrafow Zeppelin z roznymi interpreterami

zeppelin_paragraphs = [
    {
        "title": "Dokumentacja",
        "interpreter": "%md",
        "code": """## Analiza MovieLens\n### Cel: top 10 filmow per gatunek\nDane: ratings (25M) + movies (62K)"""
    },
    {
        "title": "Sprawdz klaster",
        "interpreter": "%sh",
        "code": """hdfs dfs -du -h /data/movielens/\necho '---'\ncurl -s spark-master:8080 | grep -o 'Workers.*'"""
    },
    {
        "title": "Zaladuj dane (PySpark)",
        "interpreter": "%spark.pyspark",
        "code": """ratings = spark.read.parquet("hdfs://namenode:9000/data/movielens/silver/ratings")\nmovies = spark.read.parquet("hdfs://namenode:9000/data/movielens/silver/movies")\nratings.createOrReplaceTempView("ratings")\nmovies.createOrReplaceTempView("movies")\nprint(f"Ratings: {ratings.count()}, Movies: {movies.count()}")"""
    },
    {
        "title": "Top filmy (SQL)",
        "interpreter": "%sql",
        "code": """SELECT m.title, m.primary_genre,\n       COUNT(*) as num_ratings,\n       ROUND(AVG(r.rating), 2) as avg_rating\nFROM ratings r JOIN movies m ON r.movie_id = m.movie_id\nGROUP BY m.title, m.primary_genre\nHAVING COUNT(*) >= 1000\nORDER BY avg_rating DESC\nLIMIT 20"""
    },
    {
        "title": "Wyslij do PostgreSQL (PySpark)",
        "interpreter": "%spark.pyspark",
        "code": """top_movies = spark.sql("SELECT * FROM top_movies_view")\ntop_movies.write.mode("overwrite").jdbc(\n    "jdbc:postgresql://postgres:5432/recommender",\n    "analytics.top_movies",\n    properties={"user": "recommender", "password": "recommender"}\n)"""
    },
    {
        "title": "Weryfikacja w PostgreSQL",
        "interpreter": "%jdbc(postgresql)",
        "code": """SELECT * FROM analytics.top_movies ORDER BY avg_rating DESC LIMIT 10"""
    }
]

print("=== Przykladowy Note w Zeppelin ===")
print("Kazdy paragraf moze uzywac innego interpretera:\n")
for i, p in enumerate(zeppelin_paragraphs, 1):
    print(f"--- Paragraph {i}: {p['title']} ---")
    print(f"Interpreter: {p['interpreter']}")
    print(f"Kod:")
    for line in p['code'].split('\n'):
        print(f"  {line}")
    print()

## 5. Tworzenie Note przez API

Zeppelin pozwala programowo tworzyc notebooki - przydatne do automatyzacji.

In [ ]:
def create_note(name, paragraphs):
    """Tworzy nowy Note w Zeppelin przez API."""
    note_body = {
        "name": name,
        "paragraphs": [
            {
                "title": p.get("title", ""),
                "text": f"{p['interpreter']}\n{p['code']}",
                "config": {
                    "enabled": True,
                    "editorMode": "ace/mode/scala" if "spark" in p["interpreter"] else "ace/mode/sql"
                }
            }
            for p in paragraphs
        ]
    }
    return note_body


# Tworzymy note do analizy MovieLens
movielens_note = create_note(
    "MovieLens/Top Movies Analysis",
    zeppelin_paragraphs
)

try:
    result = zep.post("/notebook", movielens_note)
    note_id = result.get("body", "")
    print(f"Utworzono Note: {note_id}")
    print(f"URL: {ZEPPELIN_URL}/#/notebook/{note_id}")
except Exception as e:
    print(f"Blad (wymaga Zeppelin): {e}")
    print(f"\nStruktura Note:")
    print(json.dumps(movielens_note, indent=2, ensure_ascii=False)[:500] + "...")

In [ ]:
# Uruchamianie paragrafow programowo
def run_paragraph(note_id, paragraph_id, wait=True, timeout=120):
    """Uruchamia paragraf i opcjonalnie czeka na wynik."""
    try:
        zep.post(f"/notebook/run/{note_id}/{paragraph_id}")
        if not wait:
            return None

        start = time.time()
        while time.time() - start < timeout:
            status = zep.get(f"/notebook/{note_id}/paragraph/{paragraph_id}")
            para_status = status.get("body", {}).get("status", "")
            if para_status in ["FINISHED", "ERROR", "ABORT"]:
                return status.get("body", {})
            time.sleep(2)
        return {"status": "TIMEOUT"}
    except Exception as e:
        return {"status": "ERROR", "error": str(e)}


def run_all_paragraphs(note_id):
    """Uruchamia caly Note."""
    try:
        zep.post(f"/notebook/job/{note_id}")
        print(f"Uruchomiono Note {note_id}")
        print(f"Sledz postep: {ZEPPELIN_URL}/#/notebook/{note_id}")
    except Exception as e:
        print(f"Blad: {e}")


# Listowanie istniejacych note
try:
    notes = zep.get("/notebook")
    print("Istniejace Notes:")
    for note in notes.get("body", []):
        print(f"  [{note.get('id', '?')}] {note.get('path', '?')}")
except Exception as e:
    print(f"Listowanie note (wymaga Zeppelin): {e}")

## 6. Wizualizacje w Zeppelin

Zeppelin oferuje wbudowane wizualizacje - wystarczy zwrocic tabelaryczne dane z `%sql` lub `%spark.sql`:

| Typ wykresu | Kiedy uzywac |
|-------------|-------------|
| Table | Przeglad surowych danych |
| Bar Chart | Porownanie kategorii |
| Pie Chart | Udzialy procentowe |
| Area Chart | Trendy w czasie |
| Line Chart | Zmiany wartosci w czasie |
| Scatter Plot | Korelacje miedzy zmiennymi |

### Angular Display System
Zeppelin Angular pozwala tworzyc interaktywne widgety z dwukierunkowym bindingiem.
Mozna budowac mini-dashboardy bez zewnetrznych narzedzi.

In [ ]:
# Przyklady wizualizacji w Zeppelin
# Te zapytania SQL automatycznie generuja wykresy w Zeppelin UI

visualization_examples = [
    {
        "title": "Bar Chart: Srednia ocena per gatunek",
        "sql": """
SELECT primary_genre, ROUND(AVG(rating), 2) as avg_rating, COUNT(*) as cnt
FROM ratings r JOIN movies m ON r.movie_id = m.movie_id
GROUP BY primary_genre
HAVING COUNT(*) > 10000
ORDER BY avg_rating DESC""",
        "chart": "Bar Chart (Keys: primary_genre, Values: avg_rating)"
    },
    {
        "title": "Line Chart: Trend ocen w czasie",
        "sql": """
SELECT rating_year, ROUND(AVG(rating), 2) as avg_rating, COUNT(*) as num_ratings
FROM ratings
WHERE rating_year BETWEEN 2000 AND 2020
GROUP BY rating_year
ORDER BY rating_year""",
        "chart": "Line Chart (Keys: rating_year, Values: avg_rating, num_ratings)"
    },
    {
        "title": "Pie Chart: Rozklad segmentow uzytkownikow",
        "sql": """
SELECT user_segment, COUNT(*) as cnt
FROM gold_user_profiles
GROUP BY user_segment""",
        "chart": "Pie Chart (Keys: user_segment, Values: cnt)"
    },
    {
        "title": "Scatter Plot: Popularnosc vs srednia ocena",
        "sql": """
SELECT title, total_ratings, avg_rating
FROM gold_movie_stats
WHERE total_ratings >= 100
LIMIT 500""",
        "chart": "Scatter (xAxis: total_ratings, yAxis: avg_rating)"
    }
]

print("=== Wizualizacje Zeppelin ===")
print("W Zeppelin te zapytania %sql automatycznie generuja wykresy:\n")
for ex in visualization_examples:
    print(f"--- {ex['title']} ---")
    print(f"Typ wykresu: {ex['chart']}")
    print(f"%sql{ex['sql']}")
    print()

print("W Zeppelin klikasz ikonke wykresu pod wynikiem i wybierasz typ.")
print("Nie trzeba matplotlib/seaborn - wbudowane wizualizacje sa interaktywne.")

## 7. Dynamic Forms - parametryzowanie notebookow

Zeppelin pozwala tworzyc interaktywne formularze w paragrafach.
Uzytkownik moze zmieniac parametry bez edycji kodu.

Dostepne typy formularzy:
- **Text input**: `${nazwa=domyslna}` - pole tekstowe
- **Select**: `${nazwa=domyslna,opt1|opt2|opt3}` - dropdown
- **Checkbox**: `${checkbox:nazwa=opt1,opt1|opt2|opt3}` - multi-select

In [ ]:
# Przyklady Dynamic Forms w Zeppelin

dynamic_form_examples = [
    {
        "title": "Filtrowanie po gatunku (dropdown)",
        "interpreter": "%sql",
        "code": """
SELECT title, avg_rating, total_ratings
FROM gold_movie_stats
WHERE primary_genre = '${genre=Drama,Drama|Comedy|Action|Thriller|Sci-Fi|Horror|Romance}'
  AND total_ratings >= ${min_ratings=100}
ORDER BY avg_rating DESC
LIMIT ${limit=20,10|20|50|100}""",
        "opis": "Uzytkownik wybiera gatunek z dropdown i ustawia min. liczbe ocen"
    },
    {
        "title": "Analiza okresu (text input)",
        "interpreter": "%sql",
        "code": """
SELECT rating_year, COUNT(*) as ratings, ROUND(AVG(rating),2) as avg
FROM ratings
WHERE rating_year BETWEEN ${year_from=2010} AND ${year_to=2020}
GROUP BY rating_year ORDER BY rating_year""",
        "opis": "Uzytkownik wpisuje zakres lat"
    },
    {
        "title": "Multi-select gatunkow (checkbox)",
        "interpreter": "%sql",
        "code": """
SELECT primary_genre, COUNT(*) as movies, ROUND(AVG(avg_rating),2) as mean_rating
FROM gold_movie_stats
WHERE primary_genre IN (${checkbox:genres=Drama,Drama|Comedy|Action|Thriller|Sci-Fi})
GROUP BY primary_genre ORDER BY movies DESC""",
        "opis": "Uzytkownik zaznacza wiele gatunkow jednoczesnie"
    }
]

print("=== Dynamic Forms w Zeppelin ===")
print("Formularze renderuja sie jako interaktywne widgety w UI:\n")
for ex in dynamic_form_examples:
    print(f"--- {ex['title']} ---")
    print(f"Opis: {ex['opis']}")
    print(f"{ex['interpreter']}")
    print(ex['code'])
    print()

print("Korzysc: analitycy moga eksplorowac dane bez znajomosci SQL!")
print("Zmiana parametru automatycznie odpala paragraf ponownie.")

## 8. Zeppelin vs Jupyter - porownanie

| Cecha | Zeppelin | Jupyter |
|-------|----------|--------|
| **Multi-interpreter** | Tak (%spark, %sql, %sh w 1 note) | Nie (1 kernel per notebook) |
| **Spark integracja** | Natywna (SparkSession gotowy) | Wymaga konfiguracji |
| **SQL** | Natywny %sql z wizualizacja | Wymaga pandas/magic |
| **Wizualizacje** | Wbudowane (klik) | matplotlib/plotly (kod) |
| **Dynamic Forms** | Wbudowane | ipywidgets (dodatkowy import) |
| **Wspolpraca** | Note sharing + permissions | JupyterHub + Git |
| **Scheduler** | Wbudowany cron | Wymaga Airflow/cron |
| **Ekosystem** | Hadoop/Spark-centric | Ogolny (ML, web, etc.) |
| **Rozszerzalnosc** | Nowy interpreter | Nowy kernel |
| **Community** | Mniejsza | Ogromna |
| **nbformat** | Wlasny JSON | .ipynb standard |

### Kiedy Zeppelin?
- Zespol pracuje z Spark/Hadoop na co dzien
- Potrzebne interaktywne dashboardy SQL
- Analitycy bez doswiadczenia z kodem
- Wymagana wspolpraca i permissions

### Kiedy Jupyter?
- Rozbudowane ML/DL pipeliny (TensorFlow, PyTorch)
- Potrzebna duza baza rozszerzen (nb_extensions)
- Preferowany format .ipynb (GitHub rendering)
- Zespol przyzwyczajony do Jupytera

In [ ]:
# Praktyczne porownanie: to samo zadanie w Zeppelin vs Jupyter

print("=== Zadanie: Top 10 filmow Drama z wizualizacja ===")

print("\n--- W Zeppelin (2 paragrafy): ---")
print("""
%sql
SELECT title, avg_rating, total_ratings
FROM gold_movie_stats
WHERE primary_genre = '${genre=Drama}' AND total_ratings >= 500
ORDER BY avg_rating DESC LIMIT 10

-> Klik na 'Bar Chart' -> GOTOWE! Interaktywny wykres + dropdown gatunku.
""")

print("\n--- W Jupyter (10+ linii kodu): ---")
print("""
import matplotlib.pyplot as plt
import pandas as pd

genre = "Drama"  # brak interaktywnego dropdown

query = f\"\"\"
SELECT title, avg_rating, total_ratings
FROM gold_movie_stats
WHERE primary_genre = '{genre}' AND total_ratings >= 500
ORDER BY avg_rating DESC LIMIT 10
\"\"\"

df = spark.sql(query).toPandas()
fig, ax = plt.subplots(figsize=(10, 6))
ax.barh(df['title'], df['avg_rating'])
ax.set_xlabel('Average Rating')
ax.set_title(f'Top 10 {genre} Movies')
plt.tight_layout()
plt.show()
""")

print("\nZeppelin wygrywa w: szybka eksploracja danych, SQL + wizualizacja")
print("Jupyter wygrywa w: customizacja wykresow, ML pipelines, reprodukowalnosc")

## 9. Wspolpraca i Permissions

Zeppelin oferuje system uprawnien na poziomie Note:

- **Owner**: pelny dostep (tworca note)
- **Writers**: moga edytowac i uruchamiac paragrafy
- **Readers**: moga ogladac wyniki (read-only)
- **Runners**: moga uruchamiac ale nie edytowac

In [ ]:
# Zarzadzanie uprawnieniami Note
def set_note_permissions(note_id, owners, writers, readers, runners=None):
    """Ustawia uprawnienia Note."""
    permissions = {
        "owners": owners,
        "writers": writers,
        "readers": readers,
        "runners": runners or []
    }
    try:
        zep.put(f"/notebook/{note_id}/permissions", permissions)
        print(f"Uprawnienia Note {note_id} zaktualizowane")
    except Exception as e:
        print(f"Blad: {e}")
    return permissions


# Model uprawnien dla systemu rekomendacji
permission_model = {
    "ETL Pipeline Monitoring": {
        "owners": ["data_engineering"],
        "writers": ["data_engineering"],
        "readers": ["data_science", "analysts"],
        "runners": ["data_engineering"]
    },
    "Movie Stats Dashboard": {
        "owners": ["data_engineering"],
        "writers": ["data_science"],
        "readers": ["analysts", "management"],
        "runners": ["analysts"]
    },
    "ML Model Training": {
        "owners": ["data_science"],
        "writers": ["data_science"],
        "readers": ["data_engineering"],
        "runners": ["data_science"]
    },
    "Business KPI Report": {
        "owners": ["analysts"],
        "writers": ["analysts"],
        "readers": ["management", "data_science", "data_engineering"],
        "runners": ["analysts", "management"]
    }
}

print("=== Model uprawnien Note w Zeppelin ===")
for note_name, perms in permission_model.items():
    print(f"\n--- {note_name} ---")
    for role, groups in perms.items():
        print(f"  {role:<10}: {', '.join(groups)}")

In [ ]:
# Scheduler - automatyczne uruchamianie Note (wbudowany cron)
def schedule_note(note_id, cron_expression):
    """Ustawia harmonogram uruchamiania Note."""
    config = {"cron": cron_expression, "releaseResource": True}
    try:
        zep.post(f"/notebook/cron/{note_id}", config)
        print(f"Harmonogram ustawiony: {cron_expression}")
    except Exception as e:
        print(f"Blad: {e}")


# Przyklady harmonogramow
schedules = {
    "ETL Pipeline Monitoring": "0 0 6 * * ?       # codziennie o 6:00",
    "Movie Stats Dashboard":   "0 0 8 * * ?       # codziennie o 8:00",
    "ML Model Training":       "0 0 2 ? * SUN     # co niedziele o 2:00",
    "Business KPI Report":     "0 0 9 ? * MON     # co poniedzialek o 9:00"
}

print("=== Harmonogramy Note (Zeppelin Cron Scheduler) ===")
for note_name, cron in schedules.items():
    print(f"  {note_name:<30} {cron}")

print("\nZeppelin uruchomi Note automatycznie wg harmonogramu.")
print("Wyniki beda widoczne w UI i moga byc eksportowane.")

## Zadanie koncowe

Stworz interaktywny dashboard rekomendacji w Zeppelin (przez API):

1. **Note: MovieLens Dashboard** z nastepujacymi paragrafami:
   - `%md` - tytul i opis dashboardu
   - `%spark.pyspark` - zaladowanie danych z HDFS (Silver ratings + movies)
   - `%sql` - top 10 filmow per gatunek (z dynamic form dropdown)
   - `%sql` - trend srednich ocen per rok (line chart)
   - `%sql` - rozklad segmentow uzytkownikow (pie chart)
   - `%sql` - scatter: popularnosc vs ocena (z dynamic form: min_ratings)
   - `%spark.pyspark` - wygeneruj rekomendacje dla user_id (z dynamic form text input)
   - `%sh` - pokaz status klastrow Spark i HDFS

2. Ustaw uprawnienia:
   - owners: data_engineering
   - writers: data_science
   - readers + runners: analysts

3. Ustaw scheduler: codziennie o 7:00

Uzyj `ZeppelinClient` do stworzenia note przez API.

In [ ]:
# Twoje rozwiazanie:


In [ ]:
# Podsumowanie
print("""
=== Podsumowanie: Apache Zeppelin w systemie rekomendacji ===

1. MULTI-INTERPRETER
   - %spark.pyspark: ETL i ML w jednym notebooku
   - %sql: interaktywne zapytania z wizualizacja
   - %sh: monitoring infrastruktury
   - %md: dokumentacja inline

2. WIZUALIZACJE
   - Wbudowane wykresy: bar, line, pie, scatter, area
   - Bez dodatkowego kodu - klik na typ wykresu
   - Angular Display: interaktywne dashboardy

3. DYNAMIC FORMS
   - Dropdown: wybor gatunku, segmentu uzytkownika
   - Text input: zakres dat, min. ocen
   - Checkbox: multi-select gatunkow
   - Analitycy eksploruja dane bez pisania kodu

4. WSPOLPRACA
   - Permissions: owner/writer/reader/runner per note
   - Scheduler: automatyczne raporty (cron)
   - Sharing: link do note z wynikami

5. ZEPPELIN vs JUPYTER
   - Zeppelin: Spark/SQL eksploracja, dashboardy, wspolpraca
   - Jupyter: ML/DL development, reprodukowalnosc, ekosystem
   - Moga wspolistniec! Zeppelin dla analitykow, Jupyter dla ML.

W naszym systemie rekomendacji:
   - Jupyter: rozwoj modelu ALS, eksperymenty ML
   - Zeppelin: dashboardy KPI, SQL eksploracja, raporty analityczne
""")